In [2]:
# ---------- 1. Install Kaggle ----------
!pip install -q kaggle

# ---------- 2. Upload kaggle.json ----------
# In Colab: Files (left) → Upload → kaggle.json
from google.colab import files
files.upload()

# ---------- 3. Configure Kaggle ----------
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# ---------- 4. Download PetImages Dataset ----------
!kaggle datasets download -d bhavikjikadara/dog-and-cat-classification-dataset

# ---------- 5. Unzip ----------
!unzip -q cat-and-dog.zip
!ls

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/dog-and-cat-classification-dataset
License(s): apache-2.0
 91% 706M/775M [00:04<00:01, 44.1MB/s]
100% 775M/775M [00:04<00:00, 200MB/s] 
unzip:  cannot find or open cat-and-dog.zip, cat-and-dog.zip.zip or cat-and-dog.zip.ZIP.
dog-and-cat-classification-dataset.zip	kaggle.json  sample_data


In [3]:
!unzip -q dog-and-cat-classification-dataset.zip


In [ ]:
# ======================================
# 1. IMPORT LIBRARIES
# ======================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from PIL import Image
import os
import matplotlib.pyplot as plt

# ======================================
# 2. DATASET CLASS (Safe Image Loading)
# ======================================
class PetDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        self.class_to_idx = {}

        classes = sorted(
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))
        )

        for idx, cls in enumerate(classes):
            self.class_to_idx[cls] = idx
            cls_path = os.path.join(root_dir, cls)
            for img in os.listdir(cls_path):
                if img.lower().endswith(('.jpg', '.png', '.jpeg')):
                    self.samples.append((os.path.join(cls_path, img), idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except:
            # Skip corrupted images
            return self.__getitem__((idx + 1) % len(self.samples))
        if self.transform:
            image = self.transform(image)
        return image, label

# ======================================
# 3. TRANSFORMS
# ======================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ======================================
# 4. LOAD DATASET & SPLIT
# ======================================
full_dataset = PetDataset("/content/PetImages")  # path to PetImages folder

total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size   = int(0.15 * total_size)
test_size  = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, val_size, test_size]
)

# Assign transforms
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform   = val_transform
test_dataset.dataset.transform  = val_transform

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))
print("Test size:", len(test_dataset))

# ======================================
# 5. DEVICE
# ======================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ======================================
# 6. MODEL (Enhanced ResNet18)
# ======================================
class EnhancedResNet18(nn.Module):
    def __init__(self, num_classes=2, dropout_rate=0.5):
        super().__init__()
        self.backbone = models.resnet18(pretrained=True)

        # Freeze backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout_rate / 2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(dropout_rate / 2),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

model = EnhancedResNet18().to(device)

# ======================================
# 7. LOSS & OPTIMIZER
# ======================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.backbone.fc.parameters(), lr=0.001)

# ======================================
# 8. TRAINING LOOP
# ======================================
epochs = 5
train_accs, val_accs = [], []

for epoch in range(epochs):
    # ---- Training ----
    model.train()
    correct = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        correct += (outputs.argmax(1) == labels).sum().item()
    train_acc = correct / len(train_dataset)
    train_accs.append(train_acc)

    # ---- Validation ----
    model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()
    val_acc = correct / len(val_dataset)
    val_accs.append(val_acc)

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

# ======================================
# 9. TESTING
# ======================================
model.eval()
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        correct += (outputs.argmax(1) == labels).sum().item()
test_acc = correct / len(test_dataset)
print(f"\nTest Accuracy: {test_acc:.4f}")

# ======================================
# 10. PLOT ACCURACY
# ======================================
plt.plot(train_accs, label="Train Accuracy")
plt.plot(val_accs, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()


Train size: 17498
Val size: 3749
Test size: 3751
Using device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 212MB/s]
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch [1/5] Train Acc: 0.9288 | Val Acc: 0.9611
